In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='jshtml')
import seaborn as sns

#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=2.2)

### Problema (Pêndulo)

<font size=3px>Um pêndulo feito de uma esfera de massa $m$ presa a um fio de massa 
    desprezível e comprimento $L$, é solto do ponto $A$, conforme mostra a figura. <br>
    a) Qual a velocidade da esfera no ponto B?<br>
    b) Qual a tração da corda quando ela está neste ponto?
</font>
<img src="pendulo.png" alt="Drawing" style="width: 250px;"/>

In [ ]:
l = 1.
m = 1.
g = 9.8

def Ftheta(t,xv):
    theta,vtheta = xv
    return -m*g*l*np.sin(theta)

    
def atheta(t,xv):
    return Ftheta(t,xv)/(m*l**2)


def RHS(t,xv):
    theta,vtheta = xv
    return [vtheta,atheta(t,xv)]

vtheta0 = 0.
theta0 = 45.*np.pi/180.
xv0 = [theta0,vtheta0]
t0, tf = 0.,5.
sol = scipy.integrate.solve_ivp(RHS,(t0,tf),xv0,rtol=1e-7,atol=1e-7)
xmax,xmin = max(sol.y[0,:]),min(sol.y[0,:])
thetaF = scipy.interpolate.interp1d(sol.t,sol.y[0,:])
vthetaF = scipy.interpolate.interp1d(sol.t,sol.y[1,:])
xF = lambda t: l*np.sin(thetaF(t))
yF = lambda t: l*(1.-np.cos(thetaF(t)))

def KF(t):
    return (m*l**2)*(vthetaF(t)**2)/2.

def UF(t):
    return m*g*l*(1.-np.cos(thetaF(t)))

In [ ]:
~tmin, tmax = t0,tf
dt = 0.05
tpts = np.linspace(tmin,tmax,100)

fig = plt.figure(figsize=(22,7))
ax1 = plt.subplot(131)
pt, = ax1.plot([xF(tmin)],[yF(tmin)],'o',markersize=25)
line, = ax1.plot([xF(tmin),0.],[yF(tmin),l],'-',linewidth=3.0)
xlabel = ax1.set_xlabel('x (m)')
ylabel = ax1.set_ylabel('y (m)')
ax1.set_xlim(-l,l)
ax1.set_ylim(-0.1*l,1.1*l)

ax2 = plt.subplot(132)
Kl, = ax2.plot(tpts,KF(tpts),'--',linewidth=4.0)
Ul, = ax2.plot(tpts,UF(tpts),'--',linewidth=4.0)
El, = ax2.plot(tpts,KF(tpts)+UF(tpts),linewidth=4.0)
xlabel = ax2.set_xlabel('t (s)')
ax2.legend(['K','U','K+U'],loc='upper right',framealpha=1.0,fontsize=25.0)


ax3 = plt.subplot(133)
Ufunc, = ax3.plot(xF(tpts),UF(tpts),'-',linewidth=4.0)
ptB, = ax3.plot([xF(tmin)],[UF(tmin)],'o',markersize=20)
eel = ax3.plot([xmin,xmax],[UF(tmin)+KF(tmin),UF(tmax)+KF(tmax)],'-',linewidth=3.)
xlabel = ax3.set_xlabel('x (m)')
ylabel = ax3.set_ylabel('U(x) (J)')


def init():
    tpts = [tmin]
    pt.set_data([xF(tmin)],[yF(tmin)])
    line.set_data([xF(tmin),0.],[yF(tmin),l])
    Kl.set_data(tpts,KF(tpts))
    Ul.set_data(tpts,UF(tpts))
    ptB.set_data([xF(tmin)],[UF(tmin)])
    return (pt,line,Kl,Ul,ptB)

def animate(i):
    
    tA = tmin+i*dt
    tpts = np.linspace(tmin,tA,100)
    xpt = [xF(tA)]
    ypt = [yF(tA)]
    pt.set_data(xpt,ypt)
    line.set_data([xF(tA),0.],[yF(tA),l])
    Kl.set_data(tpts,KF(tpts))
    Ul.set_data(tpts,UF(tpts))
    ptB.set_data([xF(tA)],[UF(tA)])
    return (pt,line,Kl,Ul,ptB)


anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=int((tmax-tmin)/dt), interval=50, 
                               blit=True)
plt.close()
anim
# plt.show()

#### Respostas:

In [ ]:
vB = np.sqrt(2*l*g*(1.-np.cos(theta0)))
TB = m*(vB**2)/l + m*g
print('a) vB = %2.3f m/s' %vB)
print('b) T = %2.3f N' %TB)